In [1]:
import os
from random import randint
import uuid

from quinine import QuinineArgumentParser
from tqdm import tqdm
import torch
import yaml
import numpy as np
from eval import get_run_metrics
from tasks import get_task_sampler
from samplers import get_data_sampler,rand_select_sampler
from curriculum import Curriculum
from schema import schema
from models import noconf_build_model
import random
import wandb

In [2]:
modelfamily = "raw"
model_n_dims = 50
n_positions = 101
n_embd = 256
n_layer = 12
n_head = 8


In [3]:
model = noconf_build_model(modelfamily,n_dims=model_n_dims, n_positions=n_positions, n_embd=n_embd, n_layer=n_layer, n_head=n_head)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0)  # add weight decay


model.cuda()
model.train()

NameError: name 'rawTransformerModel' is not defined

In [ ]:
taskname = 'ode_ivp_case2vec'
dataname= 'ode_ivp_case2'
If_enhanced_loss = False
n_dims = 50
n_points = 41
bsize = 64

In [ ]:
data_sampler = get_data_sampler(dataname,n_dims=n_dims)
xs = data_sampler.sample_xs(
    n_points,
    bsize,
)
torch.save(xs, f'test_{dataname}_xs.pt')

In [ ]:
# xs = torch.load(f'test_{dataname}_xs.pt')

In [ ]:
def train_step(model, xs, ys, optimizer, loss_func): # 单个批次（batch）的前向传播、损失计算、反向传播和优化器更新
    optimizer.zero_grad()
    output = model(xs, ys)
    loss = 2
    # loss = loss_func(output, ys)
    # loss.backward()
    # optimizer.step()
    # return loss.detach().item(), output.detach()
    return loss, output.detach()

In [ ]:
# 检查checkpoints pt

In [ ]:
print(taskname)
y_task = get_task_sampler(taskname,n_dims=n_dims,batch_size=bsize)
task_sampler = y_task()
ys = task_sampler.evaluate(xs)  # 获取模型预测结果

ode_ivp_case2vec


In [ ]:
if If_enhanced_loss:
            # 获取steptable
            steptable = task_sampler.get_training_steptable(xs,use_h = False).cuda()
            # 增强损失 每个数据点计算前几个的损失
            loss_func = task_sampler.get_enhanced_training_metric(steptable)  # 损失metric
            point_wise_loss_func = task_sampler.get_enhanced_metric(steptable)  # loss_func
else:
    loss_func = task_sampler.get_training_metric()  # 损失metric
    point_wise_loss_func = task_sampler.get_metric()  # loss_func
def mean_squared_error(ys_pred, ys):
    return (ys).square().mean()
loss_func = mean_squared_error

In [ ]:
loss, output = train_step(model, xs, ys, optimizer, loss_func)

RuntimeError: shape '[64, 41, 1]' is invalid for input of size 131200

In [ ]:
print(loss)
print(output.size())

2
torch.Size([64, 50])
